# Artificial intelligence for Ionospheric perturbations DEtections (AIDE)
AIDE allows you to scan vTEC records to detect co-seismic ionospheric perturbations. AIDE software consists of a Machine-Learning based detector, arrival-time picker, and an associator across GNSS satellite networks.

Reference paper: Brissaud, Q., & Astafyeva, E. (2021). Near-real-time detection of co-seismic ionospheric disturbances using machine learning. Geophysical Journal International. doi: 10.1093/gji/ggac167

<center><img src="scheme_ML_improved.png" width="600"/></center>
Figure: AIDE workflow

In [5]:
%matplotlib inline
import numpy as np
from pdb import set_trace as bp
import pandas as pd
import seaborn as sns

import AIDE, detector, compute_params_waveform, read_data, train_est, train_wave_picker, utils_paper, associator, constants

## Directories and data/model loading
First, select where vTEC data are located and where figures should be generated

In [6]:
## Directories
main_dir = '/staff/quentin/Documents/Projects/ML_TEC/'
dir_dict = {
    'DIR_FIGURES': main_dir + 'figures/',
    #'DIR_DATA': '/adhocdata/infrasound/2023_ML_TEC/data/'
    'DIR_DATA': main_dir + 'data/'
}

Then, add entries to `load_dict` in order to load data and/or models instead of re-generating them. Comment line or remove entry to re-generate.

In [7]:
## Loading previously-generated data/models
load_dict = {
    'est': main_dir + 'figures/model_arr4_noise4_snr1_5.0_pFalse/forest_est_s500.0_m0.7_n1000.0_w720.0.pkl',
    'data': main_dir + 'figures/model_arr4_noise4_snr1_5.0_pFalse/data_s500.0_m0.7_n1000.0_w720.0.pkl',
    'features': main_dir + 'figures/model_arr4_noise4_snr1_5.0_pFalse/features_features_m0.7_w720.0.csv',
    'features-picker': '/adhocdata/infrasound/2023_ML_TEC/data/features_picker_w720.0_d0.7.csv',
    'est-picker': main_dir + 'figures/random_est_picker_w720.0_d0.7_bTrue.joblib',
    'load_tec_data': False
    #'detections': main_dir + 'figures/detected_arrivals_Iquique_afterreview_detrend.csv',
    #'probas': main_dir + 'figures/probas_all_waveforms_Iquique_afterreview_detrend.csv',
    #'associations': main_dir + 'Iquique_afterreview_associations.csv',
    #'associations_time_steps': main_dir + 'Iquique_afterreview_associations_time_steps.csv',
}

## Create an AIDE model instance
To create a new AIDE model instance, you need to: 1) load vTEC data, 2) create an instance of the ML detector, and 3) create an instance of the ML picker

In [10]:
import importlib, AIDE, train_est
importlib.reload(AIDE)
importlib.reload(train_est)

## Create detection model
one_model = AIDE.AIDE(dir_dict)
one_model.load_data(load_dict)
one_model.load_RF_detector(load_dict)
one_model.load_RF_picker(load_dict)

Start reading data
Data quality check
- Checking param for:  0084 G15 Kii_30s
- Checking param for:  0085 G15 Kii_30s
- Checking param for:  0487 G15 Kii_30s
- Checking param for:  0492 G15 Kii_30s
- Checking param for:  0716 G15 Kii_30s
- Checking param for:  0718 G15 Kii_30s
- Checking param for:  0719 G15 Kii_30s
- Checking param for:  0720 G15 Kii_30s
- Checking param for:  0723 G15 Kii_30s
- Checking param for:  0728 G15 Kii_30s
- Checking param for:  0729 G15 Kii_30s
- Checking param for:  1049 G15 Kii_30s
- Checking param for:  1050 G15 Kii_30s
- Checking param for:  1057 G15 Kii_30s
- Checking param for:  1059 G15 Kii_30s
- Checking param for:  1080 G15 Kii_30s
- Checking param for:  1082 G15 Kii_30s
- Checking param for:  1083 G15 Kii_30s
- Checking param for:  1084 G15 Kii_30s
- Checking param for:  1085 G15 Kii_30s
- Checking param for:  1086 G15 Kii_30s
- Checking param for:  1087 G15 Kii_30s
- Checking param for:  1088 G15 Kii_30s
- Checking param for:  1089 G15 Kii_30s
- 

## Run the detector and arrival-time picker
Run the detector with input parameters defined in `detection_options` over satellite network and events defined in `detection_network`.
- `nb_picks` (int): Number of picks to use to confirm a detections
- `plot_probas` (bool): Whether or not you want to plot the detection probabilities for each input vTEC waveform
- `focus_on_arrival` (bool): Whether or not you want to only perform detections around the true detection time. You need `tec_data_param` entries for this.
- `focus_arrival_shift` (s, float): If `focus_on_arrival=True`, time window used to extract waveforms
- `nb_CPU` (int): Number of CPUs to deploy detections using multiprocessing

In [ ]:
import importlib, train_est
importlib.reload(train_est)

events = ['Kii_30s']

## Perform detection
detection_network = {
    'name': 'test',
    'events': events,
    'satellites': one_model.tec_data.loc[one_model.tec_data['event'].isin( events ), 'satellite'].unique().tolist(),
    'stations': one_model.tec_data.loc[one_model.tec_data['event'].isin( events ), 'station'].unique().tolist()[:5],
    'time_end': one_model.tec_data.loc[one_model.tec_data['event'].isin( events ), 'time_s'].max()
}

detection_options = { 
    'nb_picks': 5, 
    'plot_probas': True, 
    'focus_on_arrival': False, 
    'focus_arrival_shift': 1000., 
    'nb_CPU': 1
}
one_model.run_detections(load_dict, detection_network, **detection_options)

In [26]:
one_model.tec_data_param

,station,epoch,time,amp-TEC,t-ampmax-TEC,ampmax-TEC,diff-t,amp,slope,satellite,doy,year,event,arrival-time
0,1084,1236,660.0,-2.4593,1238,-2.3835,60.0,0.0758,0.001263,G15,249,4,Kii_30s,37080.0000
1,1050,1234,600.0,-2.6098,1235,-2.5797,30.0,0.0301,0.001003,G15,249,4,Kii_30s,37019.9988
2,0728,1240,780.0,-2.4332,1242,-2.3367,60.0,0.0965,0.001608,G15,249,4,Kii_30s,37199.9988
3,0718,1239,750.0,-2.4864,1241,-2.4404,60.0,0.0460,0.000767,G15,249,4,Kii_30s,37170.0000
4,1086,1236,660.0,-2.4935,1239,-2.4221,90.0,0.0714,0.000793,G15,249,4,Kii_30s,37080.0000
5,1089,1238,720.0,-2.4630,1240,-2.4133,60.0,0.0497,0.000828,G15,249,4,Kii_30s,37140.0012
6,0085,1233,570.0,-2.6563,1236,-2.6003,90.0,0.0560,0.000622,G15,249,4,Kii_30s,36990.0000
7,1090,1237,690.0,-2.5035,1240,-2.3990,90.0,0.1045,0.001161,G15,249,4,Kii_30s,37109.9988
8,1080,1235,630.0,-2.5156,1237,-2.4817,60.0,0.0339,0.000565,G15,249,4,Kii_30s,37050.0012
9,0723,1238,720.0,-2.5706,1242,-2.4573,120.0,0.1133,0.000944,G15,249,4,Kii_30s,37140.0012


## Run the associator
Run the associator with the inputs parameters provided in `association_options`.
- `max_radius_search` (m, float): Maximum distance between two ionospheric points for association
- `velocity_search_max` (km/s, float): Maximum horizontal propagation velocity between two ionospheric points for association
- `velocity_search_min` (km/s, float): Minimum horizontal propagation velocity between two ionospheric points for association
- `save_associations` (bool): Whether or not you want to save the `DataFrame` containing associated arrivals
- `association_name` (str): Name for this association dataset

In [ ]:
## Create association list
association_options = {
    'max_radius_search': 500., 
    'velocity_search_max': 50., 
    'velocity_search_min': 0.65, 
    'save_associations': True, 
    'association_name': 'test'
}
one_model.run_association(load_dict, **association_options)

In [ ]:
## Create Figure
first_detections = utils_paper.create_arrival_time_plot(one_model.detections, one_model.options, offset=500., nb_pts_picker=5, quantile_threshold=0.8)
utils_paper.plot_image_iono(one_model.tec_data, one_model.tec_data_param, first_detections, one_model.options, associations=one_model.associations, add_fault=False, add_inset_fault=False, unknown='slip', rotation=25., vmin=6., vmax=11., offset_source_lat=8., offset_source_lon=8., first_label='d', hion_dict={}, add_new_waveform={}, add_new_waveform_class={}, ext_name='_test')